In [ ]:
import torch
import matplotlib.pyplot as plt
from distribution import distribution
import numpy as np
%matplotlib inline

In [ ]:
def Mat(Lvec):
    N = Lvec.size(0)
    Mask = Lvec.repeat(N,1)
    Same = (Mask==Mask.t())
    return Same.clone().fill_diagonal_(0), ~Same#same diff

def distribution(data_dict, src_model='CAR_R50_result_r64_lam1.0/'):
    
    traFvec, tradset = featuralize('CAR', data_dict['tra'], src_model, 'R50', 64)
    valFvec, valdset = featuralize('CAR', data_dict['val'], src_model, 'R50', 64)
    
    traLab = torch.LongTensor([v for k,v in tradset.idx_to_class.items()])
    valLab = torch.LongTensor([v for k,v in valdset.idx_to_class.items()])
    N_tra = traFvec.size(0)
    N_val = valFvec.size(0)
    
    intv = 111
    # tra dset
    S,D = Mat(traLab)
    D_tra_tra = traFvec.mm(torch.t(traFvec))
    D_tra_tra[torch.eye(N_tra).bool()]=-1
    tt_max,tt_idx = D_tra_tra.max(1)
    
    H_tra_same = torch.histc(D_tra_tra[S], bins=intv, min=-1, max=1)/D_tra_tra[S].size(0)
    H_tra_diff = torch.histc(D_tra_tra[D], bins=intv, min=-1, max=1)/D_tra_tra[D].size(0)

    D_tra_tra_S = D_tra_tra.clone()
    D_tra_tra_S[D]=-1
    tt_max_S = D_tra_tra_S.max(1)[0]
    
    D_tra_tra_D = D_tra_tra.clone()
    D_tra_tra_D[S]=-1
    tt_max_D = D_tra_tra_D.max(1)[0]
    
    H_tra_same_max = torch.histc(tt_max_S, bins=intv, min=-1, max=1)/tt_max_S.size(0)
    H_tra_diff_max = torch.histc(tt_max_D, bins=intv, min=-1, max=1)/tt_max_D.size(0)
    
    # val dset    
    S,D = Mat(valLab)
    D_val_val = valFvec.mm(torch.t(valFvec))
    D_val_val[torch.eye(N_val).bool()]=-1
    vv_max,vv_idx = D_val_val.max(1)
    
    H_val_same = torch.histc(D_val_val[S], bins=intv, min=-1, max=1)/D_val_val[S].size(0)
    H_val_diff = torch.histc(D_val_val[D], bins=intv, min=-1, max=1)/D_val_val[D].size(0)

    D_val_val_S = D_val_val.clone()
    D_val_val_S[D]=-1
    vv_max_S = D_val_val_S.max(1)[0]
    
    D_val_val_D = D_val_val.clone()
    D_val_val_D[S]=-1
    vv_max_D = D_val_val_D.max(1)[0]
    
    H_val_same_max = torch.histc(vv_max_S, bins=intv, min=-1, max=1)/vv_max_S.size(0)
    H_val_diff_max = torch.histc(vv_max_D, bins=intv, min=-1, max=1)/vv_max_D.size(0)
    
    # val-tra
    D_val_tra = valFvec.mm(traFvec.t())
    vt_max,vt_idx = D_val_tra.max(1)
    
    return (H_tra_same,
            H_tra_diff,
            H_val_same,
            H_val_diff,
            H_tra_same_max,
            H_tra_diff_max,
            H_val_same_max,
            H_val_diff_max,
            vv_max,
            vt_max,
            (valLab==valLab[vv_idx]),
            (traLab==traLab[tt_idx]),
            vv_max_S,
            vv_max_D,
            tt_max_S,
            tt_max_D)

In [ ]:
H0 = distribution(src = 'CAR_R50_result_r64_lam1.0/HN/D64_lr0.005/2/')
H1 = distribution(src = 'CAR_R50_result_r64_lam1.0/SCT/D64_lr0.005/2/')

In [ ]:
s1 = 15
r1 = 0.3
a = 0.5
i = 29

plt.figure(figsize=(4,4))
plt.scatter(H0[14].numpy(),(H0[15]*(H0[11].float())).numpy(),s=0.2,alpha=a,label='Same Label')
plt.scatter(H0[14].numpy(),(H0[15]*(1-H0[11].float())).numpy(),s=0.2,alpha=a,label='Different Label')
plt.ylabel('Similarity',fontsize=s1)
plt.xlabel('Similarity',fontsize=s1)
plt.xlim(r1,1)
plt.ylim(r1,1)
plt.title('SHN(train)',fontsize=18)
plt.legend(loc=3,markerscale=20)
plt.savefig('tra2tra_vn'+v,dpi=200,bbox_inches='tight')